# LawGPT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install accelerate # kakaobrain/kogpt 사용에 필요함.
# !pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
# import pytorch_lightning as pl

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
transformers.logging.set_verbosity_error()

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from argparse import ArgumentParser

# import wandb
# from pytorch_lightning.loggers import WandbLogger
# wandb_logger = WandbLogger(name="LawGPT", project="LawGPT")

parser = ArgumentParser(description="LawGPT")
parser.add_argument('--text_pretrained_model', default="kogpt2", type=str)
parser.add_argument('--text_len', default=100, type=int)
parser.add_argument('--learning_rate', default=0.00005, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=10, type=int)
parser.add_argument('--seed', default=826, type=int)
args = parser.parse_args('')

# wandb.config.update(args)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

text_len = args.text_len
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
SEED = args.seed

def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # pl.seed_everything(SEED)

set_seeds()

device

device(type='cuda')

## run.py

In [4]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

if args.text_pretrained_model == "polyglot-ko":
    text_pretrained_model = "EleutherAI/polyglot-ko-1.3b"
    tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model,
                                              bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK)
    model = AutoModelForCausalLM.from_pretrained(text_pretrained_model)

if args.text_pretrained_model == "kogpt6b-ryan1.5b":
    text_pretrained_model = "kakaobrain/kogpt"
    tokenizer = AutoTokenizer.from_pretrained(
        'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
        bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
    )
    model = AutoModelForCausalLM.from_pretrained(
        'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
        pad_token_id=tokenizer.eos_token_id,
        torch_dtype='auto', low_cpu_mem_usage=True
    )

if args.text_pretrained_model == "kogpt2":
    text_pretrained_model = "skt/kogpt2-base-v2"
    tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model,
                                              bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK)
    model = AutoModelForCausalLM.from_pretrained(text_pretrained_model)   

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/precedent_preprocess.csv")
df.head()

,판시사항,판결요지
0,[1] 매도인이 악의인 계약명의신탁의 명의수탁자로부터 명의신탁의 목적물인 주택을 임...,[1] 매도인이 악의인 계약명의신탁에서 명의수탁자로부터 명의신탁의 목적물인 주택을 ...
1,임차인이 주택임대차보호법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도 임...,"주택임대차보호법 제6조, 제6조의3 등 관련 규정의 내용과 체계, 입법 취지 등을 ..."
2,[1] 대지에 관한 저당권 설정 후 건물이 신축되고 그 신축건물에 다시 저당권이 설...,[1] 대지에 관한 저당권 설정 후에 비로소 건물이 신축되고 그 신축건물에 대하여 ...
3,전세권과 임차권의 차이점 추신영 동방문화사전세권과 주택임대차의 대항력과 우선변제권 ...,- 전세권과 임차권의 차이점 (판례강의) 계약법 / 동방문화사 2015 오경미 채권...
4,가. 구 주택임대차보호법 (1981.3.5. 법률 제3379호) 부칙 제2항 단서에...,가. 구 주택임대차보호법 (1981.3.5. 법률 제3379호) 부칙 제2항 단서는...


## preprocessing.py

In [6]:
train_num = round(len(df)*0.7)
val_num = round(len(df)*0.2)
test_num = round(len(df)*0.1)

train_num, val_num, test_num

(270, 77, 39)

In [7]:
X_train = df["판시사항"].iloc[:train_num]
y_train = df["판결요지"].iloc[:train_num]

X_val= df["판시사항"].iloc[train_num:train_num+val_num]
y_val = df["판결요지"].iloc[train_num:train_num+val_num]

X_test = df["판시사항"].iloc[train_num+val_num:]
y_test = df["판결요지"].iloc[train_num+val_num:]

X_train.shape, X_val.shape, X_test.shape

((270,), (77,), (39,))

### Example

In [8]:
model.to(device='cuda', non_blocking=True)
_ = model.eval()

sample_idx = 1  # 예제 번호

prompt = X_train[sample_idx][:text_len]
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=text_len)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print("*** True Data ***")
print(X_train[sample_idx])

print("*** Input Data ***")
print(X_train[sample_idx][:text_len])

print("*** Generated Data ***")
print(generated)

*** True Data ***
임차인이 주택임대차보호법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도 임대인이나 같은 법 제3조 제4항에 따라 임대인의 지위를 승계한 임차주택의 양수인이 같은 법 제6조 제1항 전단에서 정한 기간 내에 제6조의3 제1항 단서 제8호에 따라 주택에 실제 거주하려고 한다는 사유를 들어 임차인의 계약갱신 요구를 거절할 수 있는지 여부(원칙적 적극)
*** Input Data ***
임차인이 주택임대차보호법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도 임대인이나 같은 법 제3조 제4항에 따라 임대인의 지위를 승계한 임차주택의 양수인이 같은 법 제6조
*** Generated Data ***
임차인이 주택임대차보호법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도 임대인이나 같은 법 제3조 제4항에 따라 임대인의 지위를 승계한 임차주택의 양수인이 같은 법 제6조 제1항 본문의 본문에 규정된 임차주택의 양수인을 임대인으로 보지 아니한다.
또한 임대인이 다른 임대주택의 양수인에게 임차주택의 양수를 청구할 때에도 임대인은 임대인으로서 임차주택의 양수인이 임대인의 지위를 승계한 임차주택의 양수인에게 임차주택의 양수


## data_loader.py

In [9]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, x_chats, y_chats, max_len=args.text_len):  # 데이터셋의 전처리를 해주는 부분
        self.x_data = x_chats
        self.y_data = y_chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self.x_data)

    def __getitem__(self, idx): # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        q = self.x_data[idx]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = self.y_data[idx]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:  #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:  # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :] # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [10]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [11]:
train_set = ChatbotDataset(X_train, y_train, max_len=args.text_len)
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [12]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
    break
print("end")

start
token_ids ====>  tensor([[    2,  9028,   739,  ..., 10032,  9436, 28623],
        [    2, 24433,  7627,  ...,   739, 15064,  7832],
        [19458, 13068,  9022,  ..., 24521, 50218, 19271],
        ...,
        [    2,  9175, 16365,  ...,    10,     4,  9175],
        [28334, 27606,  9249,  ...,  7416,  7743,  9554],
        [ 8340,  8213,  8213,  ...,  8758,  8346,  9833]])
mask =====>  tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])
label =====>  tensor([[    9,     9,     9,  ...,  9436, 28623,     3],
        [    9,     9,     9,  ..., 15064,  7832,     3],
        [    9,     9,     9,  ..., 50218, 19271,     3],
        ...,
        [    9,     9,     9,  ...,     9,  9175,     3],
        [    9,     9,     9,  ...,  7743,  9554,     3],
        [    9,     9,     9,  ...,  8346,  9833,     3]])
end


## model.py

In [13]:
model.to(device)
model.train()

learning_rate = args.learning_rate
criterion = nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Sneg = -1e18

## main.py

In [14]:
print("start")
print("")

for epoch in range(EPOCHS):
    print("epoch: ", epoch)
    for batch_idx, samples in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        token_ids, mask, label = [t.to(device) for t in samples]
        out = model(token_ids)
        out = out.logits # Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
    print("loss: ", avg_loss)

print("")
print("end")

start

epoch:  0


9it [00:07,  1.13it/s]


loss:  tensor(14.9341, device='cuda:0', grad_fn=<DivBackward0>)
epoch:  1


9it [00:08,  1.08it/s]


loss:  tensor(12.9049, device='cuda:0', grad_fn=<DivBackward0>)
epoch:  2


...

loss:  tensor(18.8570, device='cuda:0', grad_fn=<DivBackward0>)
epoch:  8


9it [00:07,  1.20it/s]


loss:  tensor(18.8013, device='cuda:0', grad_fn=<DivBackward0>)
epoch:  9


9it [00:07,  1.20it/s]


loss:  tensor(12.1994, device='cuda:0', grad_fn=<DivBackward0>)

end


In [15]:
# import datetime as dt

# now = dt.datetime.now()
# nowDate = now.strftime('%m%d_%H%M')

# torch.save(model.state_dict(), 'models/' + str(nowDate) + '_dict.bin')
# torch.save(model, 'models/' + str(nowDate) + '_model.bin')

In [16]:
X_train[1]

'임차인이 주택임대차보호법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도 임대인이나 같은 법 제3조 제4항에 따라 임대인의 지위를 승계한 임차주택의 양수인이 같은 법 제6조 제1항 전단에서 정한 기간 내에 제6조의3 제1항 단서 제8호에 따라 주택에 실제 거주하려고 한다는 사유를 들어 임차인의 계약갱신 요구를 거절할 수 있는지 여부(원칙적 적극)'

In [17]:
y_train[1]

'주택임대차보호법 제6조, 제6조의3 등 관련 규정의 내용과 체계, 입법 취지 등을 종합하여 보면, 임차인이 같은 법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도, 임대인으로서는 특별한 사정이 없는 한 같은 법 제6조 제1항 전단에서 정한 기간 내라면 제6조의3 제1항 단서 제8호에 따라 임대인이 목적 주택에 실제 거주하려고 한다는 사유를 들어 임차인의 계약갱신 요구를 거절할 수 있고, 같은 법 제3조 제4항에 의하여 임대인의 지위를 승계한 임차주택의 양수인도 그 주택에 실제 거주하려는 경우 위 갱신거절 기간 내에 위 제8호에 따른 갱신거절 사유를 주장할 수 있다고 보아야 한다.'

In [18]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 임차인이 주택임대차보호법 제6조의3 제1항 본문에 따라 계약갱신을 요구하였더라도 임대인이나 같은 법 제3조 제4항에 따라 임대인의 지위를 승계한 임차주택의 양수인이 같은 법 제6조 제1항 전단에서 정한 기간 내에 제6조의3 제1항 단서 제8호에 따라 주택에 실제 거주하려고 한다는 사유를 들어 임차인의 계약갱신 요구를 거절할 수 있는지 여부(원칙적 적극)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:9                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/gpt2/modeling_gpt2.py:1076 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1073 │   │   """                                                                               │
│   1074 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1075 │   │                                                                                     │
│ ❱ 1076 │   │   transformer_outputs = self.transformer(                                           │
│   1077 │   │   │   input_ids,                                                                    │
│   1078 │   │   │   past_key_values=past_key_values,                                              │
│   1079 │   │   │   attention_mask=attention_mask,                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/gpt2/modeling_gpt2.py:900 in forward │
│                                                                                                  │
│    897 │   │   │   │   │   encoder_attention_mask,                                               │
│    898 │   │   │   │   )                                                                         │
│    899 │   │   │   else:                                                                         │
│ ❱  900 │   │   │   │   outputs = block(                    

In [19]:
print("result")
print(a.strip())

result
주택임대차보호법 제6조의3 제1항  구 주택임대차보호법 제6조의3 등<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>